<a href="https://colab.research.google.com/github/saulobritto/bioinfo-tools-gcolab/blob/main/dash_bio_gcolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://dash.plotly.com/dash-bio

In [ ]:
!pip install dash-bio==0.4.8

In [ ]:
import dash_bio
print(dash_bio.__version__)

In [ ]:
!pip install dash==1.12.0

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

#AlignmentViewer

In [16]:
%%writefile app.py
import six.moves.urllib.request as urlreq
from six import PY3

import dash
import dash_bio as dashbio
import dash_html_components as html


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'alignment_viewer_p53.fasta'
).read()

if PY3:
    data = data.decode('utf-8')

app.layout = html.Div([
    dashbio.AlignmentChart(
        id='my-alignment-viewer',
        data=data
    ),
    html.Div(id='alignment-viewer-output')
])


@app.callback(
    dash.dependencies.Output('alignment-viewer-output', 'children'),
    [dash.dependencies.Input('my-alignment-viewer', 'eventDatum')]
)
def update_output(value):
    if value is None:
        return 'No data.'
    return str(value)


if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting app.py


In [11]:
get_ipython().system_raw('./ngrok http 8050 &')

In [12]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ccfc0c6293e7.ngrok.io


In [17]:
!python app.py

  File "app.py", line 25
    tilewidth = 50
            ^
SyntaxError: invalid syntax


#Circos

In [ ]:
%%writefile app2.py

import json
import six.moves.urllib.request as urlreq
from six import PY3

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'circos_graph_data.json'
).read()

if PY3:
    data = data.decode('utf-8')

circos_graph_data = json.loads(data)

app.layout = html.Div([
    dashbio.Circos(
        id='my-dashbio-circos',
        layout=circos_graph_data['GRCh37'],
        selectEvent={"0": "hover", "1": "click", "2": "both"},
        tracks=[{
            'type': 'CHORDS',
            'data': circos_graph_data['chords'],
            'config': {
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        }]
    ),
    "Graph type:",
    dcc.Dropdown(
        id='histogram-chords',
        options=[
            {'label': x, 'value': x}
            for x in ['histogram', 'chords']
        ],
        value='chords'
    ),
    "Event data:",
    html.Div(id='circos-output')
])


@app.callback(
    dash.dependencies.Output('circos-output', 'children'),
    [dash.dependencies.Input('my-dashbio-circos', 'eventDatum')]
)
def update_output(value):
    if value is not None:
        return [html.Div('{}: {}'.format(v.title(), value[v]))
                for v in value.keys()]
    return 'There are no event data. Click or hover on a data point to get more information.'


@app.callback(
    dash.dependencies.Output('my-dashbio-circos', 'tracks'),
    [dash.dependencies.Input('histogram-chords', 'value')],
    state=[dash.dependencies.State('my-dashbio-circos', 'tracks')]
)
def change_graph_type(value, current):

    if value == 'histogram':

        current[0].update(
            data=circos_graph_data['histogram'],
            type='HISTOGRAM'
        )

    elif value == 'chords':
        current[0].update(
            data=circos_graph_data['chords'],
            type='CHORDS',
            config={
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        )
    return current


if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
get_ipython().system_raw('./ngrok http 8050 &')

In [ ]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!python app2.py

#OncoPrint

In [18]:
%%writefile app3.py

import json
import six.moves.urllib.request as urlreq
from six import PY3

import dash
import dash_bio as dashbio
import dash_html_components as html


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'oncoprint_dataset3.json'
).read()

if PY3:
    data = data.decode('utf-8')

data = json.loads(data)

app.layout = html.Div([
    dashbio.OncoPrint(
        id='my-dashbio-oncoprint',
        data=data
    ),
    html.Div(id='oncoprint-output')
])


@app.callback(
    dash.dependencies.Output('oncoprint-output', 'children'),
    [dash.dependencies.Input('my-dashbio-oncoprint', 'eventDatum')]
)
def update_output(event_data):
    if event_data is None or len(event_data) == 0:
        return 'There are no event data. Hover over or click on a part \
        of the graph to generate event data.'

    event_data = json.loads(event_data)

    return [
        html.Div('{}: {}'.format(
            key,
            str(event_data[key]).replace('<br>', '\n')
        ))
        for key in event_data.keys()]


if __name__ == '__main__':
    app.run_server(debug=True)

Writing app3.py


In [19]:
get_ipython().system_raw('./ngrok http 8050 &')

In [20]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ccfc0c6293e7.ngrok.io


In [21]:
!python app3.py

Running on http://127.0.0.1:8050/
Debugger PIN: 211-640-605
 * Serving Flask app "app3" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 759-792-970
